***GENERATE DATA***

In [5]:
import cv2
import csv
import mediapipe as mp

def generate_data(label, num_samples):
    label_name_mapping = {'p': '1', 'r': '2', 's': '3'}
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands

    # For webcam input:
    cap = cv2.VideoCapture(0)
    # Get the width and height of the video capture
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        
        # Open the CSV file for writing
        if label =='p':
            csv_file = open("paper.csv", "a", newline="")
            csv_writer = csv.writer(csv_file)
        elif label =='r':
            csv_file = open("rock.csv", "a", newline="")
            csv_writer = csv.writer(csv_file)
        else:
            csv_file = open("scissors.csv", "a", newline="")
            csv_writer = csv.writer(csv_file)
        
        samples_written = 0  # Counter for samples written to CSV file
        
        while cap.isOpened() and samples_written < num_samples:
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # Draw the rectangle on the frame
            rectangle_width = 500
            rectangle_height = 500
            cv2.rectangle(image, (width - rectangle_width, 0), (width, rectangle_height), (255, 255, 255), 2)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Check if hand is within the specified rectangle
                    hand_rect = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * width
                    if hand_rect > width - rectangle_width:
                        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                        # Extract and write the hand landmarks coordinates to the CSV file
                        coordinates = []
                        label_name = label_name_mapping.get(label)
                        coordinates.append(label_name)
                        for landmark in hand_landmarks.landmark:
                            # Remove parentheses and quotation marks
                            x = landmark.x
                            y = landmark.y
                            coordinates.append(x)
                            coordinates.append(y)
                        csv_writer.writerow(coordinates)
                        
                        samples_written += 1  # Increment the counter
                
                # Display the number of samples collected
            text = f"Samples collected: {samples_written} / {num_samples}"
            cv2.putText(image, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.imshow('MediaPipe Hands', image)
            
            if cv2.waitKey(1) == ord('q'):
                break

        # Close the CSV file
        csv_file.close()

    cap.release()
    cv2.destroyAllWindows()



In [11]:
generate_data('r', 1000)

***DATA PROCESSING***

In [9]:
import pandas as pd

In [10]:
for field in ['paper', 'rock', 'scissors']:
    df = pd.read_csv(f"/Users/anhhung/Desktop/TaiLieu/DPL/r_p_s/RPS/{field}.csv")
    df.columns = df.columns.str.strip()  # Remove leading and trailing whitespace from column names
    lx = [f'x{i}' for i in range(21)]
    ly = [f'y{i}' for i in range(21)]
    # Get the values of x0 and y0
    x0 = df["x0"]
    y0 = df["y0"]

    # Adjust the values of x0, x1, ..., x20
    df.loc[:, lx] = df.loc[:, lx].sub(x0, axis=0)

    # Adjust the values of y0, y1, ..., y20
    df.loc[:, ly] = df.loc[:, ly].sub(y0, axis=0)

    # Round the values to 5 decimal places
    df.loc[:, lx + ly] = df.loc[:, lx + ly].round(5)

    # Save the modified CSV file
    df.to_csv(f"processed_{field}.csv", index=False)
